In [ ]:
import pandas as pd

data = pd.read_csv('profiles.csv')

#EDA

#print(data.head())
#print(data.describe())
#print(data.info())
#print(data.shape)
#print(data.duplicated())
#data.isnull().mean().sort_values(ascending=False)  # for percentage
#data.isnull().sum().sort_values(ascending=False)

#fill null essays with empty string
essay_cols = [col for col in data.columns if 'essay' in col]
data[essay_cols] = data[essay_cols].fillna('')

#drop 3 records without height (was thinking about filling those values with the mean, but since there are only 3 rows, decided to drop them)
data = data.dropna(subset=['height'])

#fill categorical columns with unkown
cat_cols = data.select_dtypes(include=['object', 'category']).columns
print(cat_cols)
data[cat_cols] = data[cat_cols].fillna('unknown')

#print(data.isnull().sum().sort_values(ascending=False))
#print(data.head())

#group body types - Group body types into slim, average, fit, plus or unknown
def group_body_types(bt):
    if bt in ['thin', 'skinny']:
        return 'slim'
    elif bt == 'average':
        return 'average'
    elif bt in ['athletic', 'fit', 'jacked']: 
        return 'fit'
    elif bt in ['a little extra', 'curvy', 'full figured', 'overweight']: 
        return 'plus'
    else:
        return 'unknown'

data.body_type = data.body_type.apply(group_body_types)

#Diet - for diet there are 2 pieces of info in this column, so I will divide into diet type and diet strictness
def fill_diet_type(diet):
    parts = diet.split(' ')
    if len(parts) == 1:
        return parts[0]
    else:
        return parts[1]
    
data['diet_type'] = data['diet'].apply(fill_diet_type)

def fill_diet_strictness(diet):
    parts = diet.split(' ')
    if len(parts) == 1 and parts[0] == 'unknown':
        return 'unknown'
    elif len(parts) == 1: 
        return 'neutral'
    else:
        return parts[0]

data['diet_strictness'] = data['diet'].apply(fill_diet_strictness)

strict_dict = {
    'strictly': 'strict',
    'mostly': 'flexible',
    'neutral': 'standard',
    'unknown': 'unknown'
}

data['diet_strictness'] = data['diet_strictness'].map(strict_dict)

#drinks - almost perfect, just changed some labels and grouped 2 fields
drinks_dict = {
    'unknown': 'unknown',
    'not at all': 'non-drinker',
    'rarely': 'light',
    'socially': 'moderate',
    'often': 'heavy',
    'very often': 'heavy',
    'desperately': 'very heavy',    
}

data.drinks = data.drinks.map(drinks_dict)

#drugs - good to go
#education - STILL NEEDS TO BE PROPER CLEANED CHECK THE UNIQUE VALUES AND GROUP THEM AFTER FINISHING CLEANING

#print(data.education.unique())

def split_education(edu): 
    if edu is None or pd.isna(edu) or edu == 'unknown' or edu == '':
        return pd.Series(['unknown', 'unknown'])
    parts = edu.split(' ', 2)
    if len(parts) == 1:
        return pd.Series(['graduated from', parts[0]])
    if len(parts) == 2:
        return pd.Series(['graduated from', parts[0] + ' ' + parts[1]])
    if len(parts) == 3:
        status = parts[0] + ' ' + parts[1]
        level = parts[2]
        return pd.Series([status, level])

data[['education_status', 'education_level']] = data['education'].apply(split_education)

map_edu = {
    'college/university': 'college',
    'space camp': 'unknown',
    'masters program': 'masters',
    'two-year college': 'college',
    'unknown': 'unknown',
    'high school': 'high school',
    'of space camp': 'unknown',
    'ph.d program': 'phd',
    'law school': 'law school',
    'med school': 'med school',
    'of college/university': 'college',
    'of high school': 'high school',
    'of ph.d program': 'phd',
    'of two-year college': 'college',
    'of med school': 'med school',
    'of masters program': 'masters',
    'of law school': 'law school'
}

edu_status_map = {
    'working on': 'in progress',
    'graduated from': 'finished',
    'unknown': 'unknown',
    'dropped out': 'dropped out'
}

data['education_level'] = data['education_level'].map(map_edu)
data['education_status'] = data['education_status'].map(edu_status_map)

#print(data.education_status.unique())
#print(data.education_level.unique())

#ethnicity
#print(data.ethnicity.unique())

def get_primary_race(race):
    if race == '' or race == 'unknown':
        return 'unknown'
    else:
        return race.split(',')[0].strip().lower()

data['ethnicity'] = data['ethnicity'].apply(get_primary_race)

#print(data.ethnicity.unique())

#job
#print(data.job.unique())

career_map = {
    'science / tech / engineering': 'STEM',
    'computer / hardware / software': 'STEM',
    
    'medicine / health': 'Healthcare',
    
    'education / academia': 'Education',
    
    'banking / financial / real estate': 'Business',
    'sales / marketing / biz dev': 'Business',
    'executive / management': 'Business',
    
    'artistic / musical / writer': 'Creative',
    'entertainment / media': 'Creative',
    
    'hospitality / travel': 'Service',
    'clerical / administrative': 'Service',
    'construction / craftsmanship': 'Service',
    
    'political / government': 'Government / Law',
    'law / legal services': 'Government / Law',
    'military': 'Government / Law',
    
    'transportation': 'Transportation',
    
    'student': 'Student',
    'unemployed': 'Unemployed',
    'retired': 'Retired',
    
    'rather not say': 'Other',
    'other': 'Other',
    'unknown': 'Other'
}

data['job'] = data['job'].map(career_map).fillna('Other')

#print(data.job.unique())

#last_online
print(data.last_online.unique())

data['last_online'] = pd.to_datetime(data['last_online'], format='%Y-%m-%d-%H-%M')
most_recent_date = data['last_online'].max()
data['last_online'] = (most_recent_date - data['last_online']).dt.days

#OPTIONAL - CONVERT TO CATEGORIES (ACTIVE, SEMI ACTIVE, NOT ACTIVE, etc...)
